In [1]:
# ──────────────────────────────────────────────────────────────────────
# Imports
# ──────────────────────────────────────────────────────────────────────
# ─── Core Libraries ───────────────────────────────────────────────────
import os
import ast
import random
from pathlib import Path

import numpy as np
import pandas as pd

# ─── Progress & Display ───────────────────────────────────────────────
from tqdm.notebook import tqdm
from IPython.display import display, clear_output

# ─── Scikit-learn ─────────────────────────────────────────────────────
from sklearn.base import clone
from sklearn.model_selection import (
    train_test_split,
    KFold,
    StratifiedKFold,
    cross_val_score
)
from sklearn.metrics import make_scorer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer

# ─── TensorFlow / Keras ───────────────────────────────────────────────
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import (
    layers,
    callbacks,
    regularizers,
    backend as K
)

# ─── SciKeras Wrapper ─────────────────────────────────────────────────
from scikeras.wrappers import KerasRegressor

# ─── Optuna ───────────────────────────────────────────────────────────
import optuna
from optuna.samplers import TPESampler
from optuna.pruners import MedianPruner

# ─── MLflow ───────────────────────────────────────────────────────────
import mlflow
import mlflow.sklearn

# ──────────────────────────────────────────────────────────────────────
# Metric helpers
# ──────────────────────────────────────────────────────────────────────
def rmsle_np(y_true, y_pred):
    y_pred = np.maximum(y_pred, 0)
    return np.sqrt(np.mean((np.log1p(y_pred) - np.log1p(y_true)) ** 2))

rmsle_scorer = make_scorer(rmsle_np, greater_is_better=False)

def rmsle_tf(y_true, y_pred):
    return keras.backend.sqrt(
        keras.losses.mean_squared_logarithmic_error(y_true, y_pred)
    )

# ──────────────────────────────────────────────────────────────────────
# Load data + feature-engineering transformer
# ──────────────────────────────────────────────────────────────────────
df = pd.read_csv("playground-series-s5e5/train.csv")
y  = df["Calories"]
X  = df.drop(columns=["Calories", "id"])

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=X["Sex"]
)

def add_features(Xdf: pd.DataFrame) -> pd.DataFrame:
    X = Xdf.copy()
    X["BMI"] = (X["Weight"] / (X["Height"] / 100.0) ** 2).round(2)
    X["Timed_Intensity"] = X["Duration"] * X["Heart_Rate"]
    X["Heart_Rate_Zone"] = X["Heart_Rate"] / (220 - X["Age"]) * 100
    X["Mifflin_Jeor_BMR"] = np.where(
        X["Sex"] == "male",
        (10 * X["Weight"]) + (6.25 * X["Height"]) - (5 * X["Age"]) + 5,
        (10 * X["Weight"]) + (6.25 * X["Height"]) - (5 * X["Age"]) - 161,
    )
    return X

feat_eng = FunctionTransformer(add_features, validate=False)

# ──────────────────────────────────────────────────────────────────────
# Pre-processor for the NN
# • numerics → StandardScaler
# • Sex → One-hot (drop first)
# We discover numeric columns **after** feat_eng at fit-time.
# ──────────────────────────────────────────────────────────────────────
def num_selector(df):
    return df.drop(columns=["Sex"]).select_dtypes(include="number").columns

preprocess_nn = ColumnTransformer(
    [
        ("num", StandardScaler(), num_selector),
        ("cat", OneHotEncoder(drop="first", sparse_output=False), ["Sex"]),
    ],
    remainder="drop",
)

# ──────────────────────────────────────────────────────────────────────
# Keras model factory  (SciKeras passes input_dim automatically)
# ──────────────────────────────────────────────────────────────────────
def build_nn(
    lr: float = 1e-3,
    n1: int = 256,
    n2: int = 128,
    n3: int = 0,                 # 0 ⇢ skip third layer
    dropout: float = 0.25,
    activation: str = "relu",
    kernel_regularizer=None,
    *, meta
) -> keras.Model:

    n_features = meta["n_features_in_"]

    inp = keras.Input(shape=(n_features,))
    x = layers.Dense(n1, activation=activation,
                     kernel_regularizer=kernel_regularizer)(inp)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(dropout)(x)

    x = layers.Dense(n2, activation=activation,
                     kernel_regularizer=kernel_regularizer)(x)

    if n3 > 0:                                    # optional 3rd hidden layer
        x = layers.Dense(n3, activation=activation,
                         kernel_regularizer=kernel_regularizer)(x)

    out = layers.Dense(1, kernel_regularizer=kernel_regularizer)(x)

    model = keras.Model(inp, out)
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=lr),
        loss="mean_squared_logarithmic_error",
        metrics=[rmsle_tf],
    )
    return model

# ──────────────────────────────────────────────────────────────────────
# 1.  Build the (parameterised) pipeline
# ──────────────────────────────────────────────────────────────────────
early_stop = callbacks.EarlyStopping(
    monitor="val_rmsle_tf",  
    mode="min",              
    patience=15,             
    restore_best_weights=True
)

nn_reg = KerasRegressor(
    model=build_nn,
    lr=1e-3, n1=256, n2=128, n3=0, activation="relu",
    dropout=0.25,
    epochs=60, batch_size=256,
    verbose=2, validation_split=0.1,
    callbacks=[early_stop],
    kernel_regularizer=None,
    random_state=42
)

pipe_nn = Pipeline([
    ("feat_eng", feat_eng),
    ("preprocess", preprocess_nn),
    ("model", nn_reg),
])

In [ ]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("Calories-NN-Optuna-v4")

bins = pd.qcut(y, 4, labels=False)                      
skf  = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

with mlflow.start_run(run_name="optuna_search"):

    def objective(trial):
        # ── hyper-param suggestions ──────────────────────────────
        l2_val = trial.suggest_float("l2", 1e-7, 1e-4, log=True)

        params = dict(
            model__lr      = trial.suggest_float("lr", 5e-4, 3e-3, log=True),
            model__n1      = trial.suggest_int("n1", 128, 320, step=32),
            model__n2      = trial.suggest_int("n2", 32, 160, step=32),
            model__n3      = trial.suggest_int("n3", 0, 128, step=32),
            model__dropout = trial.suggest_float("dropout", 0.10, 0.45),
            model__activation = trial.suggest_categorical(
                                  "activation", ["relu", "gelu", "selu"]),
            model__epochs  = trial.suggest_int("epochs", 40, 90),
            model__batch_size = trial.suggest_categorical(
                                  "batch_size", [128, 256, 512]),
            model__kernel_regularizer = regularizers.l2(l2_val),
            model__verbose = 0                       # keep CV quiet
        )

        pipe = clone(pipe_nn).set_params(**params)

        # ── nested MLflow run for the trial ──────────────────────
        with mlflow.start_run(nested=True):
            cv_score = cross_val_score(
                pipe, X, y,
                scoring=rmsle_scorer,
                cv=skf.split(X, bins),
                n_jobs=1
            ).mean()

            mlflow.log_metric("cv_rmsle", -cv_score)
            mlflow.log_param("l2", l2_val)
            mlflow.log_params({k: v for k, v in params.items()
                               if k != "model__kernel_regularizer"})
        return cv_score                                  # negative RMSLE

    study = optuna.create_study(
        direction="maximize",
        sampler=TPESampler(multivariate=True, group=True),
        pruner=MedianPruner(n_warmup_steps=6)
    )
    study.optimize(objective, n_trials=150, show_progress_bar=True)

    mlflow.log_metric("best_cv_rmsle", -study.best_value)

print("Best 5-fold RMSLE :", -study.best_value)

In [2]:
# ------------------------------------------------------------
# 1.  Manually declare the best hyper-parameters
#     (use the *SciKeras* prefixed names here)
# ------------------------------------------------------------
best_params = {
    "model__lr"      : 9.629857254117444e-04,
    "model__n1"      : 256,
    "model__n2"      : 32,
    "model__n3"      : 64,                 # third hidden layer size (≠0 switches it on)
    "model__dropout" : 0.17253464644475364,
    "model__activation": "gelu",
    "model__epochs"  : 86,
    "model__batch_size": 512,
    "model__verbose" : 0,                  # later you can overwrite with 2
}

# --- handle the L2 value ------------------------------------
l2_val = 1.273307277060016e-07
best_params["model__kernel_regularizer"] = regularizers.l2(l2_val)

# optional tweaks for OOF / final training
best_params["model__validation_split"] = 0.10   # internal val for EarlyStopping
# best_params["model__verbose"] = 2            # uncomment if you want epoch logs

# ------------------------------------------------------------
# 2.  Build a fresh clone of your pipeline and plug the params
# ------------------------------------------------------------
pipe_final = clone(pipe_nn).set_params(**best_params)

In [3]:
# ──────────────────────────────────────────────────────────────────────
#  OOF PREDICTIONS & VAL SCORE — Neural Net
# ──────────────────────────────────────────────────────────────────────
FE_VERSION = "v5_hrzone_bmr"            # just a tag
EXPERIMENT  = "Calories-NN-OOF-VAL-V2"  # MLflow experiment

# ── 1.  Prepare CV splitter ───────────────────────────────────────────
bins = pd.qcut(y, 4, labels=False)      # same binning used in Optuna
skf  = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

oof_nn     = np.empty(len(X))
fold_table = []

# ── 2.  MLflow setup ──────────────────────────────────────────────────
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment(EXPERIMENT)

early_stop = callbacks.EarlyStopping(
    monitor="val_rmsle_tf",              # <-- same metric Optuna used
    mode="min",
    patience=15,
    restore_best_weights=True
)

with mlflow.start_run(run_name="nn_oof_fit",
                      tags={"fe_version": FE_VERSION}):
    
    for fold, (tr_idx, val_idx) in enumerate(tqdm(skf.split(X, bins), total=5), 1):
        X_tr, X_val = X.iloc[tr_idx], X.iloc[val_idx]
        y_tr, y_val = y.iloc[tr_idx], y.iloc[val_idx]

        # ── 3.  Fresh model for this fold ────────────────────────────
        # clone the base pipeline and inject the best hyper-params
        fold_params = {
            **best_params,                              # Optuna winners
            "model__callbacks": [early_stop],           # early stopping
        }
        # add split / verbosity ONLY if not already in best_params
        fold_params.setdefault("model__validation_split", 0.10)
        fold_params.setdefault("model__verbose",          0   )

        fold_model = clone(pipe_nn).set_params(**fold_params)

        # ── 4.  Fit + predict ───────────────────────────────────────
        fold_model.fit(X_tr, y_tr)
        preds = np.maximum(fold_model.predict(X_val), 0)   # clip negatives
        oof_nn[val_idx] = preds

        # ── 5.  Fold metric & logging ───────────────────────────────
        fold_rmsle = rmsle_np(y_val, preds)
        mlflow.log_metric(f"fold{fold}_rmsle", fold_rmsle)

        fold_table.append({"fold": fold, "rmsle": fold_rmsle})
        clear_output(wait=True)
        display(pd.DataFrame(fold_table))

        K.clear_session()                                  # free GPU / RAM

    # ── 6.  Overall CV result + save artefacts ───────────────────────
    cv_rmsle = np.mean([row["rmsle"] for row in fold_table])
    mlflow.log_metric("cv_rmsle", cv_rmsle)
    print(f"5-fold CV RMSLE: {cv_rmsle:.5f}")

    Path("oof").mkdir(exist_ok=True)
    np.save("oof/oof_nn.npy", oof_nn)
    mlflow.log_artifact("oof/oof_nn.npy", artifact_path="oof")

,fold,rmsle
0,1,0.060875
1,2,0.061175
2,3,0.060545
3,4,0.060204
4,5,0.060441


5-fold CV RMSLE: 0.06065
🏃 View run nn_oof_fit at: http://127.0.0.1:5000/#/experiments/26/runs/f9c607cdac70411fb361a705df00c463
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/26


In [16]:
from math import sqrt, log1p
from mlflow.sklearn import SERIALIZATION_FORMAT_PICKLE

# 1️⃣ build a fresh pipeline
full_fit_params = {
    **best_params,                 # Optuna’s choices
    "model__validation_split": 0.0,
    "model__callbacks":   [],      # strip EarlyStopping et al.
    "model__verbose":     2        # progress bar is handy
}
pipe_final = clone(pipe_nn).set_params(**full_fit_params)

# 2️⃣ load the entire training file
df = pd.read_csv("playground-series-s5e5/train.csv")
y  = df["Calories"]
X  = df.drop(columns=["Calories", "id"])

# 3️⃣ fit + log
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("Calories-NN-Final-V5")

with mlflow.start_run(run_name="nn_full_fit_v5"):
    pipe_final.fit(X, y)

    # resubstitution RMSLE (informational)
    preds = np.maximum(pipe_final.predict(X), 0)
    train_rmsle = sqrt(np.mean((np.log1p(preds) - np.log1p(y))**2))
    mlflow.log_metric("train_rmsle", train_rmsle)

    mlflow.sklearn.log_model(
        pipe_final,
        artifact_path="model",
        serialization_format=SERIALIZATION_FORMAT_PICKLE,  # ← fix
    )
    print(f"Full-data model logged (train RMSLE {train_rmsle:.5f}) ✅")

Epoch 1/86
1465/1465 - 6s - loss: 0.2241 - rmsle_tf: 0.2044 - 6s/epoch - 4ms/step
Epoch 2/86
1465/1465 - 5s - loss: 0.0123 - rmsle_tf: 0.0761 - 5s/epoch - 3ms/step
Epoch 3/86
1465/1465 - 5s - loss: 0.0101 - rmsle_tf: 0.0698 - 5s/epoch - 3ms/step
Epoch 4/86
1465/1465 - 5s - loss: 0.0093 - rmsle_tf: 0.0672 - 5s/epoch - 3ms/step
Epoch 5/86
1465/1465 - 5s - loss: 0.0085 - rmsle_tf: 0.0645 - 5s/epoch - 3ms/step
Epoch 6/86
1465/1465 - 5s - loss: 0.0080 - rmsle_tf: 0.0625 - 5s/epoch - 3ms/step
Epoch 7/86
1465/1465 - 5s - loss: 0.0076 - rmsle_tf: 0.0607 - 5s/epoch - 3ms/step
Epoch 8/86
1465/1465 - 5s - loss: 0.0072 - rmsle_tf: 0.0592 - 5s/epoch - 3ms/step
Epoch 9/86
1465/1465 - 5s - loss: 0.0069 - rmsle_tf: 0.0579 - 5s/epoch - 3ms/step
Epoch 10/86
1465/1465 - 5s - loss: 0.0067 - rmsle_tf: 0.0569 - 5s/epoch - 3ms/step
Epoch 11/86
1465/1465 - 5s - loss: 0.0064 - rmsle_tf: 0.0557 - 5s/epoch - 3ms/step
Epoch 12/86
1465/1465 - 5s - loss: 0.0062 - rmsle_tf: 0.0546 - 5s/epoch - 3ms/step
Epoch 13/86
1

INFO:tensorflow:Assets written to: C:\Users\afise\AppData\Local\Temp\tmpgxqsi8vg\assets
2025/05/31 18:40:25 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\afise\AppData\Local\Temp\tmpnio6t0u2\model\model.pkl, flavor: sklearn). Fall back to return ['scikit-learn==1.5.2']. Set logging level to DEBUG to see the full traceback. 
2025/05/31 18:40:25 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Full-data model logged (train RMSLE 0.06146) ✅
🏃 View run nn_full_fit_v5 at: http://127.0.0.1:5000/#/experiments/30/runs/e8a1c6b9d07240b6949378f144147d3e
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/30


In [17]:
# LOADING XGBOOST FINAL FROM MLFLOW  ──────────────────────────────────────────────
mlflow.set_tracking_uri("http://127.0.0.1:5000")
exp = mlflow.get_experiment_by_name("Calories-XGB-OOF-FINAL-GOLDEN")

runs = mlflow.search_runs(
        experiment_ids=[exp.experiment_id],
        filter_string='tags.mlflow.runName = "final_rmsle_model_optuna_v4"',
        order_by=["start_time DESC"],
        max_results=1,
)
final_run_id = runs.iloc[0].run_id
print("Run found:", final_run_id)
xgb_final = mlflow.sklearn.load_model(f"runs:/{final_run_id}/model")

Run found: d5175c90bf844565ac5343aba7984ff4


In [9]:
# LOADING CATBOOST FINAL FROM MLFLOW  ──────────────────────────────────────────────
exp = mlflow.get_experiment_by_name("Calories-CB-OOF-FINAL")

runs = mlflow.search_runs(
        experiment_ids=[exp.experiment_id],
        filter_string='tags.mlflow.runName = "catboost_final_fit_v2"',
        order_by=["start_time DESC"],
        max_results=1,
)
final_run_id = runs.iloc[0].run_id
print("Run found:", final_run_id)
cb_final = mlflow.sklearn.load_model(f"runs:/{final_run_id}/model")

Run found: 4a9b55aebbce4b4f9d87ecd98a596d45


In [12]:
# ────────────────────────────────────────────────────────────────────
# 0.  Imports & helpers
# ────────────────────────────────────────────────────────────────────
import mlflow, numpy as np, pandas as pd, joblib
from pathlib import Path

TEST_CSV = "playground-series-s5e5/test.csv"
SUB_PATH = "submission_blend_v3.csv"

def download_artifact(run_id: str, artifact_path: str) -> Path:
    """Download `artifact_path` from MLflow run → returns local Path."""
    local = mlflow.artifacts.download_artifacts(run_id=run_id,
                                                artifact_path=artifact_path)
    return Path(local)

# ────────────────────────────────────────────────────────────────────
# 1.  Load the ridge meta-learner from MLflow (latest run)
# ────────────────────────────────────────────────────────────────────
META_EXP = "Calories-Stacked-Ridge"         # ← name you logged above
mlflow.set_tracking_uri("http://127.0.0.1:5000")
exp = mlflow.get_experiment_by_name(META_EXP)
assert exp, f"Experiment {META_EXP!r} not found."

meta_run = mlflow.search_runs(
    experiment_ids=[exp.experiment_id],
    filter_string='tags.mlflow.runName = "ridge_meta"',
    order_by=["start_time DESC"],
    max_results=1,
).iloc[0]

ridge_path = download_artifact(meta_run.run_id, "model/meta_ridge.pkl")
meta_model = joblib.load(ridge_path)
print("✅ Ridge blender loaded (alpha =", meta_model.alpha_, ")")

# ────────────────────────────────────────────────────────────────────
# 2.  Load test data once
# ────────────────────────────────────────────────────────────────────
df_test = pd.read_csv(TEST_CSV)
ids      = df_test["id"]
X_test   = df_test.drop(columns=["id"])

# ────────────────────────────────────────────────────────────────────
# 3.  Individual model predictions
# ────────────────────────────────────────────────────────────────────
pred_xgb = np.maximum(xgb_final.predict(X_test), 0)
pred_cb  = np.maximum(cb_final .predict(X_test), 0)
pred_nn  = np.maximum(pipe_final.predict(X_test), 0)     # keras pipeline

# stack in **the same order** used for training the meta-learner!
X_meta_test = np.column_stack([pred_xgb, pred_cb, pred_nn])

# ────────────────────────────────────────────────────────────────────
# 4.  Blend with the ridge meta-learner
# ────────────────────────────────────────────────────────────────────
blend_pred = np.maximum(meta_model.predict(X_meta_test), 0)

# ────────────────────────────────────────────────────────────────────
# 5.  Write submission file
# ────────────────────────────────────────────────────────────────────
submission = pd.DataFrame({"id": ids, "Calories": blend_pred})
submission.to_csv(SUB_PATH, index=False)
print(f"📄 Submission saved to {SUB_PATH}    →  shape {submission.shape}")


✅ Ridge blender loaded (alpha = 1000.0 )
489/489 - 1s - 532ms/epoch - 1ms/step
📄 Submission saved to submission_blend_v3.csv    →  shape (250000, 2)


In [13]:
# ────────────────────────────────────────────────────────────────────
# 0.  Imports & helpers
# ────────────────────────────────────────────────────────────────────
import mlflow, numpy as np, pandas as pd, joblib
from pathlib import Path

TEST_CSV = "playground-series-s5e5/test.csv"
SUB_PATH = "submission_blend_xgb_cb.csv"

def download_artifact(run_id: str, artifact_path: str) -> Path:
    """Download `artifact_path` from MLflow run → returns local Path."""
    local = mlflow.artifacts.download_artifacts(run_id=run_id,
                                                artifact_path=artifact_path)
    return Path(local)

# ────────────────────────────────────────────────────────────────────
# 1.  Load the *two-feature* ridge meta-learner
# ────────────────────────────────────────────────────────────────────
META_EXP = "Calories-Stacked-XGB+CB"        # <- new experiment name
mlflow.set_tracking_uri("http://127.0.0.1:5000")
exp = mlflow.get_experiment_by_name(META_EXP)
assert exp is not None, f"Experiment {META_EXP!r} not found."

meta_run = mlflow.search_runs(
    experiment_ids=[exp.experiment_id],
    filter_string='tags.mlflow.runName = "ridge_meta_xgb_cb"',   # <- run name
    order_by=["start_time DESC"],
    max_results=1,
).iloc[0]

ridge_path = download_artifact(meta_run.run_id, "model/meta_ridge_xgb_cb.pkl")
meta_model = joblib.load(ridge_path)
print(f"✅ Ridge blender loaded   (alpha = {meta_model.alpha_:.4g})")

# ────────────────────────────────────────────────────────────────────
# 2.  Load test data once
# ────────────────────────────────────────────────────────────────────
df_test = pd.read_csv(TEST_CSV)
ids      = df_test["id"]
X_test   = df_test.drop(columns=["id"])

# ────────────────────────────────────────────────────────────────────
# 3.  Base-model predictions  (XGB  +  CB)
# ────────────────────────────────────────────────────────────────────
pred_xgb = np.maximum(xgb_final.predict(X_test), 0)
pred_cb  = np.maximum(cb_final .predict(X_test), 0)

# stack **exactly in the order used for training**:  [XGB, CB]
X_meta_test = np.column_stack([pred_xgb, pred_cb])

# ────────────────────────────────────────────────────────────────────
# 4.  Blend with the ridge meta-learner
# ────────────────────────────────────────────────────────────────────
blend_pred = np.maximum(meta_model.predict(X_meta_test), 0)

# ────────────────────────────────────────────────────────────────────
# 5.  Write submission file
# ────────────────────────────────────────────────────────────────────
submission = pd.DataFrame({"id": ids, "Calories": blend_pred})
submission.to_csv(SUB_PATH, index=False)
print(f"📄 Submission saved ➜  {SUB_PATH}   (shape {submission.shape})")

✅ Ridge blender loaded   (alpha = 0.0001)
📄 Submission saved ➜  submission_blend_xgb_cb.csv   (shape (250000, 2))


In [18]:
# ────────────────────────────────────────────────────────────────────
# 0.  Imports & helpers
# ────────────────────────────────────────────────────────────────────
import mlflow, numpy as np, pandas as pd, joblib
from pathlib import Path

TEST_CSV = "playground-series-s5e5/test.csv"
SUB_PATH = "submission_blend_xgb_nn.csv"

def download_artifact(run_id: str, artifact_path: str) -> Path:
    """Download `artifact_path` from MLflow run → returns local Path."""
    local = mlflow.artifacts.download_artifacts(run_id=run_id,
                                                artifact_path=artifact_path)
    return Path(local)

# ────────────────────────────────────────────────────────────────────
# 1.  Load the *two-feature* ridge meta-learner   (XGB + NN)
# ────────────────────────────────────────────────────────────────────
META_EXP = "Calories-Stacked-XGB+NN"              # ← experiment name
mlflow.set_tracking_uri("http://127.0.0.1:5000")
exp = mlflow.get_experiment_by_name(META_EXP)
assert exp is not None, f"Experiment {META_EXP!r} not found."

meta_run = mlflow.search_runs(
    experiment_ids=[exp.experiment_id],
    filter_string='tags.mlflow.runName = "ridge_meta_xgb_nn"',   # ← run name
    order_by=["start_time DESC"],
    max_results=1,
).iloc[0]

ridge_path = download_artifact(meta_run.run_id, "model/meta_ridge_xgb_nn.pkl")
meta_model = joblib.load(ridge_path)
print(f"✅ Ridge blender loaded   (alpha = {meta_model.alpha_:.4g})")

# ────────────────────────────────────────────────────────────────────
# 2.  Load test data once
# ────────────────────────────────────────────────────────────────────
df_test = pd.read_csv(TEST_CSV)
ids      = df_test["id"]
X_test   = df_test.drop(columns=["id"])

# ────────────────────────────────────────────────────────────────────
# 3.  Base-model predictions  (XGB  +  NN)
#     – assumes `xgb_final`   and `nn_final` are already in memory
#       (load them from MLflow if not)
# ────────────────────────────────────────────────────────────────────
pred_xgb = np.maximum(xgb_final.predict(X_test), 0)
pred_nn  = np.maximum(pipe_final.predict(X_test).squeeze(), 0)

# stack **exactly in the order used for training**:  [XGB, NN]
X_meta_test = np.column_stack([pred_xgb, pred_nn])

# ────────────────────────────────────────────────────────────────────
# 4.  Blend with the ridge meta-learner
# ────────────────────────────────────────────────────────────────────
blend_pred = np.maximum(meta_model.predict(X_meta_test), 0)

# ────────────────────────────────────────────────────────────────────
# 5.  Write submission file
# ────────────────────────────────────────────────────────────────────
submission = pd.DataFrame({"id": ids, "Calories": blend_pred})
submission.to_csv(SUB_PATH, index=False)
print(f"📄 Submission saved ➜  {SUB_PATH}   (shape {submission.shape})")


✅ Ridge blender loaded   (alpha = 1000)
489/489 - 1s - 536ms/epoch - 1ms/step
📄 Submission saved ➜  submission_blend_xgb_nn.csv   (shape (250000, 2))
